<a href="https://colab.research.google.com/github/dustinkeller/Biostatistics-Notebooks/blob/main/ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oneway Analysis of Y by X


In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype
import math

from scipy import stats
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_name = 'Hot Dogs.xlsx'

file_path = f"/content/drive/MyDrive/{file_name}"

data = pd.read_excel(file_path)

data[['Taste', 'Sodium']]

,Taste,Sodium
0,Bland,495
1,Bland,477
2,Bland,425
3,Medium,322
4,Medium,482
5,Medium,587
6,Medium,370
7,Medium,322
8,Medium,479
9,Medium,375


## Define Functions

In [ ]:
def r_squared(*groups):
  all_data = pd.concat(groups)
  mean_all = np.mean(all_data)
  sst = np.sum((all_data - mean_all)**2)

  ssb = 0
  for grp in groups:
    ssb += len(grp) * (np.mean(grp) - mean_all)**2

  return ssb / sst

def adj_r_squared(*groups):
  rsquared = r_squared(*groups)
  all_data = pd.concat(groups)

  k = len(groups)

  N = len(all_data)

  return 1 - ((1-rsquared)*(N-1)/(N - k - 1))

def rmse(*groups):
  ssw = 0
  for grp in groups:
    ssw += np.sum((grp - np.mean(grp))**2)

  N = len(pd.concat(groups))
  k = len(groups)

  return np.sqrt(ssw/(N-k))

## Oneway Anova

### Summary of Fit

In [ ]:
grouped_data = [data[data['Taste'] == grp]['Sodium'] for grp in data['Taste'].unique()]

In [ ]:
sof_data = {
    'Rsquare': r_squared(*grouped_data),
    'Adj Rsquare': adj_r_squared(*grouped_data),
    'Root Mean Square Error': rmse(*grouped_data),
    'Mean of Response': np.mean(pd.concat(grouped_data)),
    'Observations': len(pd.concat(grouped_data)),
}

sof = pd.Series(data=sof_data)
sof = sof.to_frame()
sof.style.set_caption("Summary of Fit")

,0
Rsquare,0.049039
Adj Rsquare,-0.008019
Root Mean Square Error,95.291718
Mean of Response,424.833333
Observations,54.000000


### Analysis of Variance

In [ ]:
f_ratio, p_value = stats.f_oneway(*grouped_data)

In [ ]:
df_total = len(pd.concat(grouped_data)) - 1
df_within = len(pd.concat(grouped_data)) - len(grouped_data)
df_between = len(grouped_data) - 1

anova_data = {
    'Source': ['Taste', 'Error', 'C. Total'],
    'DF': [len(grouped_data) - 1, len(pd.concat(grouped_data)) - len(grouped_data), len(pd.concat(grouped_data)) - 1],
    'Sum of Squares':[0, 0, '-'],
    'Mean Square': [0, 0, '-'],
    'F Ratio': [f_ratio, '-' , '-'],
    'Prob > F': [p_value, '-', '-']
}

#Sum of Squares

#Treatment
for grp in grouped_data:
  anova_data['Sum of Squares'][0] += len(grp) * (np.mean(grp) - np.mean(pd.concat(grouped_data)))**2

#Error
for grp in grouped_data:
  anova_data['Sum of Squares'][1] += len(grp) * np.var(grp)

#Total
anova_data['Sum of Squares'][2] = sum(anova_data['Sum of Squares'][0:2])


#Mean Square
anova_data['Mean Square'][0] = anova_data['Sum of Squares'][0]/(len(grouped_data)-1)
anova_data['Mean Square'][1] = anova_data['Sum of Squares'][1]/(len(pd.concat(grouped_data)) - len(grouped_data))


anova = pd.DataFrame(data=anova_data)
anova.style.set_caption("Analysis of Variance")

,Source,DF,Sum of Squares,Mean Square,F Ratio,Prob > F
0,Taste,2,23881.410256,11940.705128,1.314982,0.277427
1,Error,51,463106.089744,9080.511564,-,-
2,C. Total,53,486987.500000,-,-,-


### Means for Oneway Anova

In [ ]:
means_data = {
    'Level': data['Taste'].unique(),
    'Number': [len(grp) for grp in grouped_data],
    'Mean': [np.mean(grp) for grp in grouped_data],
    'Std Error': [rmse(*grouped_data)/np.sqrt(len(grp)) for grp in grouped_data],
    'Lower 95%': [stats.t.interval(
        0.95, #Confidence
        len(pd.concat(grouped_data)) - len(grouped_data), #Deg freedom
        np.mean(grp), #Grp mean
        rmse(*grouped_data)/np.sqrt(len(grp)))[0] #Grp std err
                  for i, grp in enumerate(grouped_data)],
    'Upper 95%': [stats.t.interval(
        0.95, #Confidence
        len(pd.concat(grouped_data)) - len(grouped_data), #Deg freedom
        np.mean(grp), #Grp mean
        rmse(*grouped_data)/np.sqrt(len(grp)))[1] #Grp std err
                  for i, grp in enumerate(grouped_data)],

}
means = pd.DataFrame(data=means_data)
means.style.set_caption("Means for Oneway Anova")

,Level,Number,Mean,Std Error,Lower 95%,Upper 95%
0,Bland,10,466.500000,30.133887,406.003697,526.996303
1,Medium,39,418.102564,15.258887,387.469070,448.736058
2,Scrumptious,5,394.000000,42.615752,308.445308,479.554692


## Means Comparisons for All Pairs Using Tukey-Kramer HSD

### Confidence Quantile

In [ ]:
q_star = (1/np.sqrt(2))*stats.studentized_range.ppf(1-0.05,len(grouped_data),df_within)

q_data = {
    # q* = (1/sqrt(2))*q, where q = Alpha percentile of studentized range distribution
    'q*': [q_star],
    'Alpha': [0.05]

}

conf_quant = pd.DataFrame(data=q_data)
conf_quant.style.set_caption("Confidence Quantile")

,q*,Alpha
0,2.413980,0.050000


### HSD Threshold Matrix

In [ ]:
MSw = anova_data['Sum of Squares'][1]/(len(pd.concat(grouped_data)) - len(grouped_data))

pooled_n = len(grouped_data[0])

np.sqrt(MSw/pooled_n) * stats.studentized_range.ppf(1-0.05,len(grouped_data),df_within)

hsd_matrix = np.zeros((3,3))

for i in range(hsd_matrix.shape[0]):
  for j in range(hsd_matrix.shape[1]):
    if i == j:
      pooled_n = 1/len(grouped_data[i])
    else:
      pooled_n = 0.5* (1/len(grouped_data[i]) + 1/len(grouped_data[j]))
    mean_dif = np.abs(np.mean(grouped_data[i]) - np.mean(grouped_data[j]))
    hsd_matrix[i,j] = mean_dif - np.sqrt(MSw*pooled_n) * stats.studentized_range.ppf(1-0.05,len(grouped_data),df_within)

hsd_frame = pd.DataFrame(data=hsd_matrix, columns=data['Taste'].unique(), index=data['Taste'].unique())
hsd_frame.style.set_caption("Abs(Dif) - HSD")

,Bland,Medium,Scrumptious
Bland,-102.873552,-33.139516,-53.493855
Medium,-33.139516,-52.092048,-85.166638
Scrumptious,-53.493855,-85.166638,-145.485172


### Ordered Differences Report

In [ ]:
import itertools

combinations = list(itertools.combinations(grouped_data, 2))
level_combinations = list(itertools.combinations(data['Taste'].unique(), 2))
index_combinations = list(itertools.combinations([i for i in range(len(combinations))], 2))

#Std Err Dif
# rmse(*grouped_data)/np.sqrt(len(combo[1])) - rmse(*grouped_data)/np.sqrt(len(combo[0]))



ord_data = {
    'Level 1': [t[0] for t in level_combinations],
    'Level 2': [t[1] for t in level_combinations],
    'Difference': [np.abs(t[0].mean() - t[1].mean()) for t in combinations],
    'Std Err Dif': [((rmse(*grouped_data)/np.sqrt(len(t[0])))**2 + (rmse(*grouped_data)/np.sqrt(len(t[1])))**2)**0.5 for t in combinations],
    'Lower CL': [0,0,0],
    'Upper CL': [0,0,0],
    'p-Value': [stats.tukey_hsd(*grouped_data).pvalue[t[0],t[1]] for t in index_combinations]
}

for i in range(len(combinations)):
  ord_data['Lower CL'][i] = ord_data['Difference'][i] - q_star * ord_data['Std Err Dif'][i]
  ord_data['Upper CL'][i] = ord_data['Difference'][i] + q_star * ord_data['Std Err Dif'][i]

ord_table = pd.DataFrame(data=ord_data)
ord_table = ord_table.sort_values('Difference', ascending=False)
ord_table.style.set_caption("Ordered Differences Report")

,Level 1,Level 2,Difference,Std Err Dif,Lower CL,Upper CL,p-Value
1,Bland,Scrumptious,72.500000,52.193424,-53.493855,198.493855,0.354110
0,Bland,Medium,48.397436,33.776986,-33.139516,129.934388,0.331889
2,Medium,Scrumptious,24.102564,45.265174,-85.166638,133.371766,0.855777


### Connecting Letters Report

In [ ]:
#Scrapped for now, will add later if necessary

# letters_data = {
#     'Level': data['Taste'].unique(),

# }

# letters = pd.DataFrame(data=letters_data)
# letters.style.set_caption("Connecting Letters Report")

,Level
0,Bland
1,Medium
2,Scrumptious
